In [1]:
import os
import argparse

import torch

import mixed_precision
from stats import StatTracker
from datasets import Dataset, build_dataset, get_dataset, get_encoder_size
from model import Model
from checkpoint import Checkpointer
from task_self_supervised import train_self_supervised
from task_classifiers import train_classifiers
from task_unsupervised import train_unsupervised

# Use encoder checkpoint to test classifier

In [3]:
training_checkpoint_dir = './checkpoint'

# get classifier training dataset
stl10_dataset = get_dataset('STL10')
stl10_encoder_size = get_encoder_size(stl10_dataset)

stl10_train_loader, stl10_test_loader, stl10_num_classes = \
    build_dataset(dataset=stl10_dataset,
                  batch_size=4,
                  labeled_only=True)

# get a helper object for tensorboard logging
stat_tracker = StatTracker(log_dir='./logs')
checkpointer = Checkpointer(training_checkpoint_dir)
torch_device = torch.device('cpu')

encoder_model_path = os.path.join(training_checkpoint_dir, 'unsup_amdim_cpt.pth')
classifier_model = checkpointer.restore_model_from_checkpoint(encoder_model_path, training_classifier=True)

train_classifier_task = train_classifiers
train_classifier_task(classifier_model, 0.0002, stl10_dataset, stl10_train_loader,
    stl10_test_loader, stat_tracker, checkpointer, training_checkpoint_dir, torch_device)

Files already downloaded and verified
Files already downloaded and verified
log_dir: ./logs
Using a 128x128 encoder
***** CHECKPOINTING *****
Model restored from checkpoint.
Self-supervised training epoch 50
Classifier training epoch 0
*************************


RuntimeError: Calculated padded input size per channel: (1 x 1). Kernel size: (3 x 3). Kernel size can't be greater than actual input size